# Core API

This notebook introduces:

- The `IVData` layout
- `tsls`/`liml`/`fuller` for workflow point estimates
- `ar_test`/`lm_test`/`clr_test` for weak-IV robust inference
- `weakiv_inference` as a unified entry point

## Implementation context (for contributors)

- What to build: a consistent, front-door API (`IVModel` + `weakiv_inference`)
  with standardized result objects.
- Why it matters: reduces friction for applied users and makes weak-IV robust
  inference visible in routine workflows.
- Literature/benchmarks: Stata weak-IV postestimation; estimatr's regression UX.
- Codex-ready tasks: add `IVModel.from_arrays`, `IVResults.weakiv`, and
  plotting helpers for p-value curves.
- Tests/docs: unit tests for API surfaces + notebook examples with pinned seeds.


In [1]:
from pathlib import Path
import ivrobust as ivr

ART = Path("artifacts") / "01_core_api"
ART.mkdir(parents=True, exist_ok=True)

ivr.set_style()

In [2]:
data, beta_true = ivr.weak_iv_dgp(n=600, k=4, strength=0.3, beta=1.25, seed=1)
data.nobs, data.p_exog, data.p_endog, data.k_instr

(600, 1, 1, 4)

## Workflow estimator: 2SLS

2SLS standard errors are conventional (strong-ID). Use AR inference if instrument strength is questionable.

In [3]:
tsls_res = ivr.tsls(data, cov_type="HC1")
tsls_res.beta, beta_true

(1.438721219093607, 1.25)

In [4]:
liml_res = ivr.liml(data, cov_type="HC1")
fuller_res = ivr.fuller(data, alpha=1.0, cov_type="HC1")
liml_res.beta, fuller_res.beta

(1.4387212190936067, 1.4426881104601441)

## Model-style API

The model wrapper mirrors familiar stats interfaces.

In [5]:
model = ivr.IVModel.from_arrays(
    y=data.y,
    x_endog=data.d,
    z=data.z,
    x_exog=None,
    add_const=True,
)
model_results = model.fit(estimator="liml", cov_type="HC1")
model_results.params.ravel()[:3]

array([0.03973068, 1.43872122])

The 2SLS estimate should be close to the true value on average in this DGP,
but its standard errors are not weak-IV robust. Use AR inference for validity
under weak identification.

## Weak-IV robust inference: AR/LM/CLR tests


In [6]:
weakiv = ivr.weakiv_inference(
    data,
    beta0=beta_true,
    alpha=0.05,
    methods=("AR", "LM", "CLR"),
    cov_type="HC1",
)
weakiv.tests["AR"], weakiv.tests["LM"], weakiv.tests["CLR"]

(TestResult(statistic=6.60419717577684, pvalue=0.15834237807728577, df=4, method='AR', cov_type='HC1', alpha=0.05, cov_config={}, warnings=(), details={'beta0': 1.25, 'nobs': 600, 'k_instr': 4}),
 TestResult(statistic=1.5386657606976417, pvalue=0.21481651330248777, df=1, method='KP-LM', cov_type='HC1', alpha=0.05, cov_config={}, warnings=(), details={'beta0': 1.25, 'nobs': 600, 'k_instr': 4}),
 TestResult(statistic=2.1238500930431883, pvalue=0.15388526049214502, df=4, method='CQLR', cov_type='HC1', alpha=None, cov_config={}, warnings=(), details={'beta0': 1.25, 'lambda1': 69.85596232711056}))

A non-rejection at the true beta is expected. Under weak instruments, the AR
test remains correctly sized, unlike conventional Wald tests.

## Confidence sets


In [7]:
cs = weakiv.confidence_sets["AR"]
cs.confidence_set.intervals

[(1.1417950052320065, 1.6544029070479105)]

The AR confidence set may be wide or disjoint in weak-ID settings. This is a
feature of weak-IV robust inference, not a numerical issue.

In [8]:
fig, ax = ivr.plot_ar_confidence_set(cs)
ivr.savefig(fig, ART / "ar_confidence_set", formats=("png", "pdf"))

[PosixPath('artifacts/01_core_api/ar_confidence_set.png'),
 PosixPath('artifacts/01_core_api/ar_confidence_set.pdf')]

## P-value curves for AR/LM/CLR

Comparing the three weak-IV robust tests on a common grid.

In [ ]:
weakiv_grid = ivr.weakiv_inference(
    data,
    beta0=beta_true,
    alpha=0.05,
    methods=("AR", "LM", "CLR"),
    cov_type="HC1",
    grid=(beta_true - 2.0, beta_true + 2.0, 301),
    return_grid=True,
)
fig, ax = weakiv_grid.plot()
ivr.savefig(fig, ART / "pvalue_curve", formats=("png", "pdf"))